In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
url='../input/data-scientist-jobs/DataScientist.csv'
data = pd.read_csv(url, encoding="utf-8")

In [ ]:
#data['Rating'].dtypes
print(data.dtypes)

In [ ]:
# Drop columns: index (1), Competitors (15), Easy Apply (16)
del data['index']
del data['Unnamed: 0']
del data['Competitors']
del data['Easy Apply']

In [ ]:
pd.set_option("display.max.columns", None)
pd.set_option("max_colwidth", 15)
pd.set_option("display.precision",2)
print(data)



In [ ]:
#Create two new columns Salary_Lo Salary_Hi, from Salary Estimate (4)
#data['Salary Estimate'][2] # $111K-$181K (Glassdoor est.)

# remove extra text
data['Salary Estimate'] = data['Salary Estimate'].str.split(' ').str[0] # all text after the space
data['Salary Estimate'] = data['Salary Estimate'].str.split('(').str[0] # in the case of no space

# split on '-' into two columns
data[['Salary_Lo','Salary_Hi']] = data['Salary Estimate'].str.split('-', n=1, expand=True)

# remove punctuation ($) and change the K into the 000 it represents
data['Salary_Lo'] = data['Salary_Lo'].str.replace('[{}]'.format(string.punctuation), '')
data['Salary_Hi'] = data['Salary_Hi'].str.replace('[{}]'.format(string.punctuation), '')
data['Salary_Lo'] = data['Salary_Lo'].str.replace('K',"000")
data['Salary_Hi'] = data['Salary_Hi'].str.replace('K',"000")

# make the columns behave as int
data["Salary_Lo"] = data['Salary_Lo'].astype('int')
data["Salary_Hi"] = data['Salary_Hi'].astype('int')

#data[['Salary Estimate','Salary_Lo','Salary_Hi']]

In [ ]:
#Create two new columns City, State/Country, from Location (8)
data[['Location_City','Location_State_Country']] = data['Location'].str.split(',', n=1, expand=True)

#data[['Location','Location_City','Location_State_Country']]

#Create two new columns City, State/Country, from Headquarters (9) 
data['Headquarters'] = data['Headquarters'].str.replace('-1','-1,-1') # take care of single values for two columns
data[['Headquarters_City','Headquarters_State_Country']] = data['Headquarters'].str.split(',', n=1, expand=True)

#data[['Headquarters','Headquarters_City','Headquarters_State_Country']]


In [ ]:
#Create two new columns Size_From, Size_To, from Size (10)
data['Size'] = data['Size'].str.replace('-1','-1 to -1') # take care of single values for two columns 
data['Size'] = data['Size'].str.replace('Unknown','-1 to -1') # replace Unknown with '-1'
data['Size'] = data['Size'].str.replace("+"," to 20000") # replace + sign with arbitrary figure
data['Size'] = data['Size'].str.split('+').str[0] # all text after the space
data[['Size_From','Size_To']] = data['Size'].str.split('to', n=1, expand=True)
data['Size_From'] = data['Size_From'].str.strip()
data['Size_To'] = data['Size_To'].str.strip()

data['Size_To'] = data['Size_To'].str.split(' ').str[0] # all text after the space

# make the columns behave as int
data["Size_From"] = data['Size_From'].astype('int')
data["Size_To"] = data['Size_To'].astype('int')
data[['Size_From','Size_To']]

In [ ]:
# normalise ['Revenue'] "Unknown / Non-Applicable" and "-1"     TOTAL:  1392 
data['Revenue'] = data['Revenue'].str.replace('Unknown / Non-Applicable','-1') # replace Unknown with '-1'


In [ ]:
# clean up company name
data['Company Name'] = data['Company Name'].str.split("\n").str[0] # all text after the \n
#data['Company Name'] = data['Company Name'].str.replace('MÃ©decins Sans FrontiÃ¨res','Médecins Sans Frontières')

In [ ]:
data['Company Name']

In [ ]:
testrow = 374
data['Size'][testrow]

In [ ]:
data['Size_From'][testrow]

In [ ]:
data['Size_To'][testrow]

In [ ]:
print (data.shape)

In [ ]:
# remove replaced columns
del data['Salary Estimate']
del data['Headquarters']
del data['Location']
del data['Size']
data.head

In [ ]:
pd.set_option("display.max.columns", 3)
pd.set_option("max_colwidth", None)
pd.set_option("display.precision", 2)
data.head()

In [ ]:
print(data.describe())

In [ ]:
industry_list = data['Industry'].unique().tolist()
sector_list = data['Sector'].unique().tolist()
company_list = data['Company Name'].unique().tolist()


In [ ]:
len(company_list)
#company_list

In [ ]:
print(len(sector_list)," sectors, ",len(industry_list), " industries and ", len(company_list), " unique companies." )

In [ ]:
data.to_csv(r'data_scientists.csv')

# Import a module to display a link to the file
from IPython.display import FileLink
# Import a module to delete the file
import os
# Create a download function
def csv_download_link(data, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv within a Jupyter notebook

    Parameters
    ----------
    df : pandas data frame
    csv_file_name : str
    delete_prompt : bool
    """
    data.to_csv(csv_file_name, index=False)
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        os.remove(csv_file_name)
# Use the function to diplay a download link
csv_download_link(data, 'data_scientists.csv') # uncomment this line 

In [ ]:
# sets lower case 
data['Job Description'] = data['Job Description'].str.lower()  

# make a set of the job_skills 
skills_list = ["java","python"," c ","scala","spark","django","golang","pig",
               "cloud","azure","aws","ec2","selenium","nlp","sas",
               "sql","hadoop","mapreduce","git","matlab","pandas"," r ",
               "amplitude","big data","tableau","powerbi","excel",
               "tensorflow","keras","pytorch","scrapy",]
education_list = ["phd","master","bachelor","graduate","md",
                  "pharmd","ba","bs","ma","ms"] # extra
db_list = ["postgresql","mysql","mssql","mongo","nosql"]
two_word_skills = ["neural network","machine learning", "data mining",
                   "map reduce", "business intelligence", "post doctoral"]
other_skills = ["wire-framing"]
job_skills = set(skills_list)
job_skills.update(education_list)
job_skills.update(db_list)

# total of occurances of two word skills in list
for idx, word in enumerate(two_word_skills):
    info = data['Job Description'].str.contains(two_word_skills[idx]).sum()
    print (idx, word, info)
    

# add the two word skill as a one word skill for the later set comparisons
# uses list to replace instances of significant word with space in between
for idx, word in enumerate(two_word_skills):           
    current_word = two_word_skills[idx]
    data['Job Description'] = data['Job Description'].str.replace(current_word,current_word.replace(" ",""))
    other_skills.append(current_word.replace(" ",""))
job_skills.update(other_skills)

# create a list column that contains skill set
words = data['Job Description'].str.lower().str.findall("\w+")

print(job_skills)

i=0
for x in words:
    newset = x                                         # sets deal with unique values nicely 
    words[i] = list(job_skills.intersection(newset))   # add the words showing up in both sets presented
    if len(words[i])==0:                               # catch an empty list passed to dataframe
        item = 'no skill noted'
        words[i].append(item)
    i = i+1  # increment row number
    
#type(x) #debug

data['Job Skills'] = words

In [ ]:
print("There are ", sum('no skill noted' in s for s in data['Job Skills']), " job postings with no discernable skills!")

In [ ]:
#print column
data['Job Skills']

In [ ]:
#verify that a particular row has the correct details
row_number = 3872
row_number = row_number - 2
print(data['Job Skills'][row_number] , data['Job Description'][row_number])

In [ ]:
# check the two word list has been taken care of - this is just for debugging and that it is to demonstrate to the marker
for idx, word in enumerate(two_word_skills):
    info = data['Job Description'].str.contains(two_word_skills[idx]).sum()
    print (idx, word, info)
    


In [ ]:
states = data['Location_State_Country'].unique().tolist()
city = data['Location_City'].unique().tolist()
states[8] = "UK"
state_set = set(states) 
city_set = set(city) 
print("There are ", (len(city_set)) ," unique cities and ", (len(state_set)) , " unique states.")
states[8]

In [ ]:
# using a pandas plot
low_salary = data.groupby('Location_State_Country')['Salary_Hi'].plot.hist(bins=11) 
high_salary = data.groupby('Location_State_Country')['Salary_Lo'].plot.hist(bins=11)

# This is difficult to comprehend

In [ ]:
# using a pandas plot

low_salary = data.groupby('Location_State_Country')['Salary_Hi'].mean().plot.hist(bins=11, stacked=True, histtype='stepfilled') 
high_salary = data.groupby('Location_State_Country')['Salary_Lo'].mean().plot.hist(bins=11, stacked=True, histtype='step')

# This is also difficult to understand

In [ ]:
# shows a comparison mean of all of the salaries in each unique state/country using matlab type plots

lowest_salary =  data.groupby('Location_State_Country')['Salary_Lo']
highest_salary =  data.groupby('Location_State_Country')['Salary_Hi']

plt.plot(states, lowest_salary.mean())
plt.plot(states, highest_salary.mean())

plt.title('Mean Highest and Lowest salary vs. state', fontsize=14)
plt.xlabel('State', fontsize=14)
plt.ylabel('Salary', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
# shows average of all of the salaries in each unique state/country as bar plot using classes

lowest_salary =  data.groupby('Location_State_Country')['Salary_Lo']
highest_salary =  data.groupby('Location_State_Country')['Salary_Hi']
labels = states

x = np.arange(len(labels))  #
width = 0.4

fig, ax = plt.subplots()
fig.set_figheight(6)
fig.set_figwidth(18)

rects1 = ax.bar(x - width/2, lowest_salary.mean(), width, label='Lowest Salary')
rects2 = ax.bar(x + width/2, highest_salary.mean(), width, label='Highest Salary')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Salary')
ax.set_title('Mean Salary In Each State')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

plt.grid(True)
plt.show()

In [ ]:
states


In [ ]:
# shows average of all of the salaries in each unique state/country as bar plot using classes

lowest_salary =  data.groupby('Location_State_Country')['Salary_Lo']
highest_salary =  data.groupby('Location_State_Country')['Salary_Hi']
labels = states

x = np.arange(len(labels))  #
width = 0.8

fig, ax = plt.subplots()
fig.set_figheight(6)
fig.set_figwidth(10)

rects1 = ax.bar(x , lowest_salary.count(), width, label='Number of Jobs')
#rects2 = ax.bar(x + width/2, highest_salary.count(), width, label='Highest Salary')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Job number')
ax.set_title('Number of jobs In Each State')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set
ax.legend()

ax.bar_label(rects1, padding=3)
#ax.bar_label(rects2, padding=3)

plt.grid(True)
plt.show()

In [ ]:
print(data.describe())

In [ ]:
# a terrible one which doesnt anything
plt.plot(data['Salary_Hi'], data['Size_To'])
plt.title('Size of Company vs. Hi Salary', fontsize=14)
plt.xlabel('Highest Salary', fontsize=14)
plt.ylabel('Company Size', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
# function for a plot word cloud
def plot_cloud(wordcloud):
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud) 
    plt.axis("off");

In [ ]:
from collections import Counter
data['Job Words'] = data['Job Skills'].apply(lambda x: ', '.join(x))
data['Job Words'] = data['Job Words'].str.replace(',', '')


In [ ]:
words = ""
for string in data['Job Words']:
    addwords = str(string)
    words = words + addwords



In [ ]:

# the word cloud package needs to be imported
from wordcloud import WordCloud, STOPWORDS

wordcloud = WordCloud(width = 1550, height = 1300, random_state=1, background_color='black', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(words)
plot_cloud(wordcloud)

In [ ]:
data['company_age'] = 2021 - data['Founded']

In [ ]:
data['company_age']

In [ ]:
data['Average_Salary'] = ((data['Salary_Hi'] + data['Salary_Lo']) /2)

In [ ]:
print("Hi ", data['Salary_Hi'][4], " Lo ", data['Salary_Lo'][4], " Average: ", data['Average_Salary'][4])

In [ ]:
import math
sns.regplot(x="company_age", y="Average_Salary", data=data);

In [ ]:
print(data.groupby('Job Title').agg({'Average_Salary':'mean'}))

In [ ]:
x = data.groupby('Location_State_Country')['Average_Salary'].mean().sort_values().tail(15)
x.plot(kind='barh')